In [175]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using Plots
include("quadruped.jl")

  Activating environment at `C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\Project.toml`


newton_solve

## Load Full Model

In [174]:
full_model = UnitreeA1()
n,m = state_dim(full_model), control_dim(full_model)
mvis = initialize_visualizer(full_model);

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8707
└ @ MeshCat C:\Users\pnadan\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73


## Load Simplified Model

In [171]:
mf = RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_foot")).mass
mb = RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "trunk")).mass + 
     RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_hip")).mass*4 +
     RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_thigh")).mass*4 +
     RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_calf")).mass*4
J = RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "trunk")).moment
state = MechanismState(full_model.mech)
zero!(state)
leg = translation(relative_transform(state, 
      default_frame(findbody(full_model.mech, "RR_hip")),
      default_frame(findbody(full_model.mech, "RR_foot"))))
s1 = translation(relative_transform(state, 
     default_frame(findbody(full_model.mech, "trunk")),
     default_frame(findbody(full_model.mech, "RR_hip"))))
s1 = vcat(-s1[2], s1[1], s1[3])
s2 = s1.*[1,-1,1]
s3 = s1.*[-1,1,1]
s4 = s1.*[-1,-1,1]

include("simple_quadruped.jl")
model = SimpleQuadruped(mf=mf, mb=mb, Jx=J[1,1], Jy=J[2,2], Jz=J[3,3], ℓmax=leg[3], s1=s1, s2=s2, s3=s3, s4=s4)
n,m = state_dim(model), control_dim(model);